In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
# load train and test dataset
def load_dataset(isCategorical=True):
    # load dataset
    data_x = np.load('data_x_aumentated_normalized.npy')
    data_y = np.load('data_y_aumentated_normalized.npy')
    #data_x = np.load('data_x_original_augmented.npy')
    #data_y = np.load('data_y_original_augmented.npy')
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
    if isCategorical == True:
        # one hot encode target values
        train_y = to_categorical(train_y)
        test_y = to_categorical(test_y)
        validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [4]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [5]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset(False)
#trainX, testX = prep_pixels(trainX, testX)

TOTAL_IMAGES =  2368
train_size =  1894
test_size =  236
validation_size =  238


In [6]:
clf1 = tf.keras.models.load_model("displasys_mobilenet-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")

In [7]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [8]:
clf2 = tf.keras.models.load_model("displasys_resnet50v2-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")

In [9]:
clf3 = tf.keras.models.load_model("resnet50-baseline-holdout-100epochs_normalized.npy")

# VGG + Flatting + Gaussian

In [10]:
clf1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228864   
 nal)                                                            
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          294944    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                        

In [15]:
trainX.shape

(1894, 128, 128, 3)

In [14]:
VGG_LR_FLATTING = GaussianNB().fit(trainX, np.ravel(trainY))

ValueError: y should be a 1d array, got an array of shape (1894, 4) instead.

In [ ]:
vgg_featureVector_flatting_test = vgg_lr_flatting.predict(testX)

In [ ]:
acc = VGG_LR_FLATTING.score(vgg_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

# MobileNet + Flatting + LogisticRegression

In [ ]:
clf2.summary()

In [ ]:
mobile_LR_FLATTING = RandomForestClassifier(n_estimators=100, random_state=1).fit(trainX, np.ravel(trainY))

In [ ]:
mobile_featureVector_flatting_test = mobile_lr_flatting.predict(testX)

In [ ]:
acc = mobile_LR_FLATTING.score(mobile_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

# Resnet + Flatting + LogisticRegression

In [ ]:
clf3.summary()

In [ ]:
resnet_LR_FLATTING = LogisticRegression(random_state=0).fit(trainX, np.ravel(trainY))

In [ ]:
resnet_featureVector_flatting_test = resnet_lr_flatting.predict(testX)

In [ ]:
acc = resnet_LR_FLATTING.score(resnet_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

# Emsemble Gaussian + RandomForest + LogisticRegression (VGG , MobileNet, Resnet)

In [ ]:
#creating the emsemble
eclf = VotingClassifier(estimators=[('lr_vgg', VGG_LR_FLATTING), ('gaussian_mobile', mobile_LR_FLATTING), ('rf_resnet', resnet_LR_FLATTING)], voting='soft')

In [ ]:
#training the emsemble with all features
eclf.fit(trainX, np.ravel(trainY))

In [ ]:
acc = eclf.score(testX, testY)
print('Acurácia: %.3f' % (acc * 100.0))